In [ ]:
from vllm import LLM, SamplingParams
import json
from tqdm import tqdm
import time

In [ ]:
llm = LLM(
    # model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    model="google/gemma-3-4b-it",
    dtype="bfloat16",
    tensor_parallel_size=1,
    gpu_memory_utilization=0.9,
    max_model_len=1024
)

# Configure generation parameters
sampling_params = SamplingParams(
    temperature=0.6,
    top_p=0.95,
    max_tokens=256  # Adjust based on your needs
)

# # Configure generation parameters
# sampling_params = SamplingParams(
#     temperature=1.0,
#     top_p=0.998,
#     max_tokens=256  # Adjust based on your needs
# )



In [ ]:
# def wrap_prompt(prompt: str) -> str:
#     return f'<｜User｜> {prompt} <｜Assistant｜>'
#
# with open("val_language.json", "r") as f:
#     prompts = json.load(f)
#
#
# example1 = "put green squash into pot or pan"  #2
# example2 = 'Pick the carrot from left side and place it on the orange cloth'  # 3
# example3 = 'open the drawer'
# example4 = 'uncover the pot and put the lid on the left side of the cloth'
#
# sentences = list(prompts.values())[:100]
#
# from collections import deque
#
# last_words1 = deque(maxlen=5)
# last_words2 = deque(maxlen=5)
#
# for lang, num_sg in sentences:
#     system_prompt = (f"I need to decompose a language command into 2 smaller language commands. "
#                      f"This language command consists of grasping an object and placing it somewhere. "
#                      f"Here are some examples.\n"
#                      f"Example 1: Decompose '{example1}' into 2 smaller language commands. Output format:\n"
#                      f"grasp the green squash.\n"
#                      f"place the green squash into the pot or pan.\n"
#                      f"Example 2: Decompose '{example2}' into 2 smaller language commands. Output format:\n"
#                      f"grab the carrot from the left side.\n"
#                      f"place the carrot on the orange cloth.\n"
#                      f"Example 3: Decompose '{example3}' into 2 smaller language commands. Output format:\n"
#                      f"grasp the drawer handling.\n"
#                      f"pull the drawer open.\n"
#                      f"Example 4: Decompose '{example4}' into 2 smaller language commands. Output format:\n"
#                      f"grab the pot lid.\n"
#                      f"put the pot lid on the left side of the cloth.\n"
#                      f"Now decompose this language command: '{lang}' into exactly 2 smaller language commands. "
#                      f"For the first command, DO NOT use any of the following verbs: {list(last_words1)}. "
#                      f"Remember the first command must be a synonym of grasp. "
#                      f"For the second command, DO NOT use any of the following verbs: {list(last_words2)}. "
#                      f"Remember the second command must be a synonym of place or pull. "
#                      f"IMPORTANT: You must ONLY output the 2 sentences with no additional text, explanations, or thinking process.")
#
#     # Approximate
#     num_tokens = len(system_prompt.split())
#     assert num_tokens < 1024
#
#     system_prompt = wrap_prompt(system_prompt)
#     #
#     outputs = llm.generate([system_prompt], sampling_params)
#
#     # Extract and store the generated text
#     generated_text = outputs[0].outputs[0].text[1:]  # for some reason, there's always a space at the beginning
#     command1, command2 = generated_text.split('\n')[:2]
#     verb1 = command1.split()[0]
#     verb2 = command2.split()[0]
#
#     last_words1.append(verb1)
#     last_words2.append(verb2)
#
#     print(lang)
#     print(command1)
#     print(command2)
#

def wrap_prompt(prompt: str) -> str:
    return f'<｜User｜> {prompt} <｜Assistant｜>'

with open("train_language.json", "r") as f:
    prompts = json.load(f)


example1 = "put green squash into pot or pan"  #2
example2 = 'Pick the carrot from left side and place it on the orange cloth'  # 3
example3 = 'open the drawer'
example4 = 'uncover the pot and put the lid on the left side of the cloth'

# If python >= 3.7, the order will match
file_names = list(prompts.keys())
sentences = list(prompts.values())

from collections import deque

batch_size = 8
last_words1 = deque(maxlen=batch_size)
last_words2 = deque(maxlen=batch_size)

decomposed_language = {}

for i in tqdm(range(0, len(sentences), batch_size)):
    fn_batch = file_names[i:i+batch_size]
    sentence_batch = sentences[i:i+batch_size]
    batch_prompts = []
    for lang, _ in sentence_batch:
        system_prompt = (f"I need to decompose a language command into 2 smaller language commands. "
                         f"This language command consists of grasping an object and placing it somewhere. "
                         f"Here are some examples.\n"
                         f"Example 1: Decompose '{example1}' into 2 smaller language commands. Output format:\n"
                         f"grasp the green squash.\n"
                         f"place the green squash into the pot or pan.\n"
                         f"Example 2: Decompose '{example2}' into 2 smaller language commands. Output format:\n"
                         f"grab the carrot from the left side.\n"
                         f"place the carrot on the orange cloth.\n"
                         f"Example 3: Decompose '{example3}' into 2 smaller language commands. Output format:\n"
                         f"grasp the drawer handling.\n"
                         f"pull the drawer open.\n"
                         f"Example 4: Decompose '{example4}' into 2 smaller language commands. Output format:\n"
                         f"grab the pot lid.\n"
                         f"put the pot lid on the left side of the cloth.\n"
                         f"Now decompose this language command: '{lang}' into exactly 2 smaller language commands. "
                         f"For the first command, DO NOT use any of the following verbs: {list(last_words1)}. "
                         f"Remember the first command must be a synonym of grasp. "
                         f"For the second command, DO NOT use any of the following verbs: {list(last_words2)}. "
                         f"Remember the second command must be a synonym of place or pull. "
                         f"IMPORTANT: You must ONLY output the 2 sentences with no additional text, explanations, or thinking process.")

        # Approximate check
        num_tokens = len(system_prompt.split())
        assert num_tokens < 1024

        batch_prompts.append(wrap_prompt(system_prompt))


    outputs = llm.generate(batch_prompts, sampling_params)

     # Process outputs
    for j, (output, fn) in enumerate(zip(outputs, fn_batch)):
        generated_text = output.outputs[0].text.strip()
        try:
            # Extract the commands
            command1, command2 = generated_text.split('\n')[:2]

            # Extract verbs
            verb1 = command1.split()[0]
            verb2 = command2.split()[0]

            # Update verb tracking
            last_words1.append(verb1)
            last_words2.append(verb2)

            # Store results
            lang, _ = sentence_batch[j]
            # print(f"Original: {lang}")
            # print(f"Command 1: {command1}")
            # print(f"Command 2: {command2}")
            # print("-" * 50)

            decomposed_language[fn] = {"original_command": lang,
                                       "subgoal_commands": [command1, command2]}

            # results.append({
            #     "original": lang,
            #     "command1": command1,
            #     "command2": command2,
            #     "verb1": verb1,
            #     "verb2": verb2
            # })
        except Exception as e:
            print(f"Error processing output {j}: {e}")
            print(f"Raw output: {generated_text}")
            print("-" * 50)
            raise e

    # print(f"Processed batch {i//batch_size + 1}/{(len(sentences) + batch_size - 1)//batch_size}")

with open ("train_decomposed_language.json", "w") as f:
    json.dump(decomposed_language, f, indent=4)


